# Initiate Study Session

#### This code simulates a user selecting a deck to study. These are the steps needed to prime the deck for today's study. 

- Determine how many cards are due for review
- Determine how many learning cards are due for study
- Determine how many new cards to add to Leitner Box

In [1]:
import pandas as pd
import sqlite3
from datetime import datetime

In [22]:
# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('flashcardsTable.db')

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

In [3]:
# # create flashcard sqlite table

# cursor.execute('''
# CREATE TABLE IF NOT EXISTS flashcardsTable (
#     id INTEGER PRIMARY KEY, 
#     native TEXT NOT NULL,
#     target TEXT NOT NULL,
#     lastStudied TEXT NOT NULL,
#     interval INTEGER NOT NULL, 
#     cardstate TEXT NOT NULL
# )
# ''')

In [4]:
# # load csv with the flashcard data and save it to sql table

# # Define the path to your CSV file
# csv_file_path = 'flashcardsLingq.csv'

# # Load the CSV into a pandas DataFrame without headers (header=None)
# # You need to manually specify the column names to match the SQLite table
# columns = ['native', 'target', 'lastStudied', 'interval', 'cardstate']  # Adjust this based on your table structure
# df = pd.read_csv(csv_file_path, header=None, names=columns)


In [6]:
# # Save the DataFrame to the SQLite table
# # 'customers' should be the name of your pre-existing SQLite table
# df.to_sql('flashcardsTable', conn, if_exists='append', index=False)

399

In [15]:
# #select first 5 rows of table to show it worked

# cursor.execute('SELECT * FROM flashcardsTable LIMIT 5')

# # Fetch all results
# rows = cursor.fetchall()

# # Option 1: Print the results directly
# print("First 5 rows:")
# for row in rows:
#     print(row)

## Load Entire Deck into Panda df and Calculate Due Field

In [23]:
# Query the entire table and store it into a pandas DataFrame
df = pd.read_sql_query("SELECT * FROM flashcardsTable", conn)
# Close the connection to the database
conn.close()
df.head(5)

,id,native,target,lastStudied,interval,cardstate
0,1,The reason for which I couldn't attend the mee...,La razón por la cual no pude asistir a la reun...,09/25/2024 01:00:00,5,UNSEEN
1,2,"A few minutes ago, I received an important call.","Hace unos minutos, recibí una llamada importante.",09/25/2024 01:00:00,5,UNSEEN
2,3,A new problem has appeared in the project.,Ha aparecido un nuevo problema en el proyecto.,09/25/2024 01:00:00,5,UNSEEN
3,4,"After finishing the exams, I decided to unleas...","Después de terminar los exámenes, decidí dar r...",09/25/2024 01:00:00,5,UNSEEN
4,5,"After he goes to another office, we will meet.","Después de que vaya a otra oficina, nos reunir...",09/25/2024 01:00:00,5,UNSEEN


In [24]:
# Convert the 'datetime_text' column to datetime type
df['lastdate'] = pd.to_datetime(df['lastStudied'])
# Add a new column that adds the 'minutes' to the 'datetime' column
df['due_datetime'] = df['lastdate'] + pd.to_timedelta(df['interval'], unit='m')
#create new columns of of last studied and due dates, ignoring the hours, minutes, and seconds
df['lastdate_calendar'] = df['lastdate'].dt.date
df['duedate_calendar'] = df['due_datetime'].dt.date

In [25]:
df.head(5)

,id,native,target,lastStudied,interval,cardstate,lastdate,due_datetime,lastdate_calendar,duedate_calendar
0,1,The reason for which I couldn't attend the mee...,La razón por la cual no pude asistir a la reun...,09/25/2024 01:00:00,5,UNSEEN,2024-09-25 01:00:00,2024-09-25 01:05:00,2024-09-25,2024-09-25
1,2,"A few minutes ago, I received an important call.","Hace unos minutos, recibí una llamada importante.",09/25/2024 01:00:00,5,UNSEEN,2024-09-25 01:00:00,2024-09-25 01:05:00,2024-09-25,2024-09-25
2,3,A new problem has appeared in the project.,Ha aparecido un nuevo problema en el proyecto.,09/25/2024 01:00:00,5,UNSEEN,2024-09-25 01:00:00,2024-09-25 01:05:00,2024-09-25,2024-09-25
3,4,"After finishing the exams, I decided to unleas...","Después de terminar los exámenes, decidí dar r...",09/25/2024 01:00:00,5,UNSEEN,2024-09-25 01:00:00,2024-09-25 01:05:00,2024-09-25,2024-09-25
4,5,"After he goes to another office, we will meet.","Después de que vaya a otra oficina, nos reunir...",09/25/2024 01:00:00,5,UNSEEN,2024-09-25 01:00:00,2024-09-25 01:05:00,2024-09-25,2024-09-25


## Determine number of cards due for review

These are cards that have been graduated from the learning phase and have also passed their interval

In [32]:
reviewCards = df[(df['cardstate'] == 'GRADUATED') & (df['duedate_calendar'] <= df['lastdate_calendar'])]
print(len(reviewCards))

0


## Determine number of learning cards due for review

These are cards are not new, not graduated, not unseen, and have met their due date

In [33]:
learningCards = df[(df['cardstate'] == 'LEARNING') & (df['duedate_calendar'] <= df['lastdate_calendar'])]
print(len(learningCards))

0


## Add NEW Cards to a total of 20

In [36]:
# Step 1: Count how many 'NEW' records there are
new_count = (df['cardstate'] == 'NEW').sum()
print(new_count)

20


In [35]:
# Step 2: If there are fewer than 20 'NEW' records, update some 'UNSEEN' records to 'NEW'
if new_count < 20:
    # Calculate how many 'UNSEEN' need to be converted to 'NEW'
    unseen_to_new = 20 - new_count
    
    # Find the indices of the first 'UNSEEN' records
    unseen_indices = df[df['cardstate'] == 'UNSEEN'].index[:unseen_to_new]
    
    # Update those 'UNSEEN' records to 'NEW'
    df.loc[unseen_indices, 'cardstate'] = 'NEW'

In [38]:
df.head(30)

,id,native,target,lastStudied,interval,cardstate,lastdate,due_datetime,lastdate_calendar,duedate_calendar
0,1,The reason for which I couldn't attend the mee...,La razón por la cual no pude asistir a la reun...,09/25/2024 01:00:00,5,NEW,2024-09-25 01:00:00,2024-09-25 01:05:00,2024-09-25,2024-09-25
1,2,"A few minutes ago, I received an important call.","Hace unos minutos, recibí una llamada importante.",09/25/2024 01:00:00,5,NEW,2024-09-25 01:00:00,2024-09-25 01:05:00,2024-09-25,2024-09-25
2,3,A new problem has appeared in the project.,Ha aparecido un nuevo problema en el proyecto.,09/25/2024 01:00:00,5,NEW,2024-09-25 01:00:00,2024-09-25 01:05:00,2024-09-25,2024-09-25
3,4,"After finishing the exams, I decided to unleas...","Después de terminar los exámenes, decidí dar r...",09/25/2024 01:00:00,5,NEW,2024-09-25 01:00:00,2024-09-25 01:05:00,2024-09-25,2024-09-25
4,5,"After he goes to another office, we will meet.","Después de que vaya a otra oficina, nos reunir...",09/25/2024 01:00:00,5,NEW,2024-09-25 01:00:00,2024-09-25 01:05:00,2024-09-25,2024-09-25
5,6,"After overcoming his fears, he no longer cared...","Después de superar sus miedos, ya no le import...",09/25/2024 01:00:00,5,NEW,2024-09-25 01:00:00,2024-09-25 01:05:00,2024-09-25,2024-09-25
6,7,"After playing, the son's mom will clean Max.","Después de jugar, la mamá del hijo limpiará a ...",09/25/2024 01:00:00,5,NEW,2024-09-25 01:00:00,2024-09-25 01:05:00,2024-09-25,2024-09-25
7,8,"After so many issues, I want to get rid of thi...","Después de tantos problemas, quiero quitarme y...",09/25/2024 01:00:00,5,NEW,2024-09-25 01:00:00,2024-09-25 01:05:00,2024-09-25,2024-09-25
8,9,"After so much practice, I know it by heart now.","Después de tanto practicar, ahora lo sé de mem...",09/25/2024 01:00:00,5,NEW,2024-09-25 01:00:00,2024-09-25 01:05:00,2024-09-25,2024-09-25
9,10,"After the explanation, it's clear to me that w...","Después de la explicación, me queda claro que ...",09/25/2024 01:00:00,5,NEW,2024-09-25 01:00:00,2024-09-25 01:05:00,2024-09-25,2024-09-25


# Deck is now ready for study

Due dates and times have been calculated, new cards have been added to the Leitner Box, and we are ready to study. The logic of which cards we study in this session is simple:

- cardstate is NEW, or
- cardstate is LEARNING, & duedate_calendar equals today, or
- cardstate is GRADUATED, & duedate_calendar equals today

# Display Next Card

In [19]:
# Close the connection to the database
conn.close()